In [1]:
clc;
clear;
% Exemplified by a read-world (7,4,3) volume tensor
% where we have 7 detectors, 4 days and 3 time windows.
tensor(:,:,1) = [155,74,493,426;108,44,350,359;...
    175,78,567,581;181,111,517,552;...
    137,53,489,485;90,44,306,290;...
    139,55,398,390];
tensor(:,:,2) = [172,69,590,386;104,39,310,304;...
    158,74,505,546;176,90,525,552;...
    150,64,438,459;73,32,281,299;...
    127,51,358,382];
tensor(:,:,3) = [225,92,443,436;94,44,355,356;...
    139,77,575,604;175,98,574,553;...
    126,67,593,484;58,49,348,301;...
    144,71,444,396];
[n1,n2,n3] = size(tensor);

In [2]:
% Create missing values for this (7,4,3) tensor randomly.
S = round(rand(n1,n2,n3)+0.3);
sparse_tensor = S.*tensor;
pos_obs = find(sparse_tensor~=0); % index set of observed entries
pos_unobs = find(sparse_tensor==0); % index set of missing entries

In [3]:
%%file cp_combination.m
function tensor = cp_combination(factor_mat,dim)
% This is a function for the CP combination over factor matrices.

d = length(dim);
factor_mat = flipud(factor_mat);
if d == 2
    tensor = factor_mat{2}*factor_mat{1}';
elseif d == 3
    mat = kr(factor_mat{1},factor_mat{2});
    tensor = mat2ten(factor_mat{d}*mat',dim,1);
else
    mat = kr(factor_mat{1},factor_mat{2});
    for k = 3:d-1
        mat = kr(mat,factor_mat{k});
    end
    tensor = mat2ten(factor_mat{d}*mat',dim,1);
end
end

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\cp_combination.m'.


In [4]:
%%file khatrirao_fast.m
function P = khatrirao_fast(varargin)
%KHATRIRAO Khatri-Rao product of matrices.
%
%   KHATRIRAO(A,B) computes the Khatri-Rao product of matrices A and
%   B that have the same number of columns.  The result is the
%   column-wise Kronecker product
%   [KRON(A(:,1),B(:,1)) ... KRON(A(:,n),B(:,n))]
%
%   KHATRIRAO(A1,A2,...) computes the Khatri-Rao product of
%   multiple matrices that have the same number of columns.
%
%   KHATRIRAO(C) computes the Khatri-Rao product of
%   the matrices in cell array C.
%
%   KHATRIRAO(...,'r') computes the Khatri-Rao product in reverse
%   order.
%
%   Examples
%   A = rand(5,2); B = rand(3,2); C = rand(2,2);
%   khatrirao(A,B) %<-- Khatri-Rao of A and B
%   khatrirao(B,A,'r') %<-- same thing as above
%   khatrirao({C,B,A}) %<-- passing a cell array
%   khatrirao({A,B,C},'r') %<-- same as above
%
%   See also TENSOR, KTENSOR.
%
%MATLAB Tensor Toolbox.
%Copyright 2012, Sandia Corporation.

% This is the MATLAB Tensor Toolbox by T. Kolda, B. Bader, and others.
% http://www.sandia.gov/~tgkolda/TensorToolbox.
% Copyright (2012) Sandia Corporation. Under the terms of Contract
% DE-AC04-94AL85000, there is a non-exclusive license for use of this
% work by or on behalf of the U.S. Government. Export of this data may
% require a license from the United States Government.
% The full license terms can be found in the file LICENSE.txt


%% Error checking on input and set matrix order
% Note that this next if/else check forces A to be a cell array.
if ischar(varargin{end}) && varargin{end} == 'r'
    if nargin == 2 && iscell(varargin{1})
        % Input is a single cell array
        A = varargin{1};
    else
        % Input is a sequence of matrices
        A = {varargin{1:end-1}};
    end
    matorder = length(A):-1:1;
else
    if nargin == 1 && iscell(varargin{1})
        % Input is a single cell array
        A = varargin{1};
    else
        % Input is a sequence of matrices
        A = varargin;
    end
    matorder = 1:length(A);
end

%% Error check on matrices and compute number of rows in result 

% N = number of columns (must be the same for every input)
N = size(A{1},2); 

% After loop, M = number of rows in the result
M = 1; 

for i = matorder
    if ndims(A) ~= 2
        error('Each argument must be a matrix');
    end
    if (N ~= size(A{i},2))
        error('All matrices must have the same number of columns.')
    end
    M = M * size(A{i},1);
end

%% Computation
P = kr(A([matorder]));

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\khatrirao_fast.m'.


In [5]:
%%file kr.m
function X = kr(U,varargin)
%KR Khatri-Rao product.
%   kr(A,B) returns the Khatri-Rao product of two matrices A and B, of 
%   dimensions I-by-K and J-by-K respectively. The result is an I*J-by-K
%   matrix formed by the matching columnwise Kronecker products, i.e.
%   the k-th column of the Khatri-Rao product is defined as
%   kron(A(:,k),B(:,k)).
%
%   kr(A,B,C,...) and kr({A B C ...}) compute a string of Khatri-Rao 
%   products A o B o C o ..., where o denotes the Khatri-Rao product.
%
%   See also kron.

%   Version: 21/10/10
%   Authors: Laurent Sorber (Laurent.Sorber@cs.kuleuven.be)

if ~iscell(U), U = [U varargin]; end
K = size(U{1},2);
% if any(cellfun('size',U,2)-K)
%     error('kr:ColumnMismatch', ...
%           'Input matrices must have the same number of columns.');
% end
J = size(U{end},1);
X = reshape(U{end},[J 1 K]);
for n = length(U)-1:-1:1
    I = size(U{n},1);
    A = reshape(U{n},[1 I K]);
    X = reshape(bsxfun(@times,A,X),[I*J 1 K]);
    J = I*J;
end
X = reshape(X,[size(X,1) K]);

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\kr.m'.


In [6]:
%%file mat2ten.m
function tensor = mat2ten(mat,dim,k)
% Folding a matrix into a tensor.
% 	dim = (n1,n2,...,nd)
% 	k=1,2,...,d

dim0 = [([dim(1:k-1),dim(k+1:length(dim))]),dim(k)];
tensor = permute(reshape(mat',dim0),[1:k-1,length(dim),k:length(dim)-1]);
end

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\mat2ten.m'.


In [7]:
%%file ten2mat.m
function mat = ten2mat(tensor,dim,k)
% This is a function for tensor unfolding.
% 	dim = (n1,n2,...,nd)
% 	k=1,2,...,d

mat = reshape(permute(tensor,[k,1:k-1,k+1:length(dim)]),dim(k),[]);
end

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\ten2mat.m'.


In [8]:
%%file BGCP_Gibbs.m
function [tensor_hat,factor_mat,final_result] = BGCP_Gibbs(dense_tensor,sparse_tensor,varargin)
% Bayesian Gaussian CP decomposition (BGCP) using Gibbs sampling.

dim = size(sparse_tensor);
d = length(dim);
position = find(sparse_tensor~=0);
pos = find(dense_tensor>0 & sparse_tensor==0);
binary_tensor = zeros(dim);
binary_tensor(position) = 1;

ip = inputParser;
ip.addParamValue('CP_rank',30,@isscalar);
ip.addParamValue('maxiter',1000,@isscalar);
ip.parse(varargin{:});

r = ip.Results.CP_rank;
maxiter = ip.Results.maxiter;

U = cell(d,1);
for k = 1:d
    U{k} = 0.1*randn(dim(k),r);
end

beta0 = 1;
nu0 = r;
mu0 = zeros(r,1);
tau_epsilon = 1;
a0 = 1;
b0 = 1;
W0 = eye(r);

%% Test Gibbs samplers for BGCP model.
rmse = zeros(maxiter,1);
fprintf('\n------Bayesian Gaussian CP decomposition using Gibbs sampling------\n');
for iter = 1:maxiter
    for k = 1:d
        % Sample hyper-parameters \Lambda^{(k)} and \mu^{(k)}.
        U_bar = mean(U{k},1)';
        var_mu0 = (dim(k)*U_bar+beta0*mu0)./(dim(k)+beta0);
        var_nu = dim(k)+nu0;
        var_W = inv(inv(W0)+dim(k)*cov(U{k})+dim(k)*beta0/(dim(k)+beta0)*(U_bar-mu0)*(U_bar-mu0)');
        var_W = (var_W+var_W')./2;
        var_Lambda0 = wishrnd(var_W,var_nu);
        var_mu0 = mvnrnd(var_mu0,inv((dim(k)+beta0)*var_Lambda0))';
        
        % Sample factor matrice U^{(k)}.
        var1 = khatrirao_fast(U{[1:k-1,k+1:d]},'r')';
        var2 = kr(var1,var1);
        var3 = reshape(var2*(ten2mat(binary_tensor,dim,k)'),[r,r,dim(k)]);
        var4 = tau_epsilon*var1*ten2mat(sparse_tensor,dim,k)'+ones(r,dim(k)).*(var_Lambda0*var_mu0);
        for i = 1:dim(k)
            var_Lambda1 = tau_epsilon*var3(:,:,i)+var_Lambda0;
            inv_var_Lambda1 = inv((var_Lambda1+var_Lambda1')./2);
            var_mu = inv_var_Lambda1*var4(:,i);
            U{k}(i,:) = mvnrnd(var_mu,inv_var_Lambda1);
        end
    end
    
    % Compute the estimated tensor.
    tensor_hat = cp_combination(U,dim);
    rmse(iter,1) = sqrt(sum((dense_tensor(pos)-tensor_hat(pos)).^2)./length(pos));
    
    % Sample precision \tau_{\epsilon}.
    var_a = a0+0.5*length(position);
    error = sparse_tensor-tensor_hat;
    var_b = b0+0.5*sum(error(position).^2);
    tau_epsilon = gamrnd(var_a,1./var_b);
    
    % Print the results.
    fprintf('iteration = %g, RMSE = %g.\n',iter,rmse(iter));
end

%% Average factor matrices over additional iterations.
fprintf('\n------Final Result of Bayesian Gaussian CP decomposition------\n');
factor_mat = cell(d,1);
for k = 1:d
    factor_mat{k} = zeros(dim(k),r);
end
tensor_hat0 = zeros(dim);
iters = 500;
for iter = 1:iters
    for k = 1:d
        % Sample hyper-parameters \Lambda^{(k)} and \mu^{(k)}.
        U_bar = mean(U{k},1)';
        var_mu0 = (dim(k)*U_bar+beta0*mu0)./(dim(k)+beta0);
        var_nu = dim(k)+nu0;
        var_W = inv(inv(W0)+dim(k)*cov(U{k})+dim(k)*beta0/(dim(k)+beta0)*(U_bar-mu0)*(U_bar-mu0)');
        var_W = (var_W+var_W')./2;
        var_Lambda0 = wishrnd(var_W,var_nu);
        var_mu0 = mvnrnd(var_mu0,inv((dim(k)+beta0)*var_Lambda0))';
        
        % Sample factor matrice U^{(k)}.
        var1 = khatrirao_fast(U{[1:k-1,k+1:d]},'r')';
        var2 = kr(var1,var1);
        var3 = reshape(var2*(ten2mat(binary_tensor,dim,k)'),[r,r,dim(k)]);
        var4 = tau_epsilon*var1*ten2mat(sparse_tensor,dim,k)'+ones(r,dim(k)).*(var_Lambda0*var_mu0);
        for i = 1:dim(k)
            var_Lambda1 = tau_epsilon*var3(:,:,i)+var_Lambda0;
            inv_var_Lambda1 = inv((var_Lambda1+var_Lambda1')./2);
            var_mu = inv_var_Lambda1*var4(:,i);
            U{k}(i,:) = mvnrnd(var_mu,inv_var_Lambda1);
        end
        factor_mat{k} = factor_mat{k}+U{k};
    end
    
    % Compute an estimated tensor.
    tensor_hat = cp_combination(U,dim);
    tensor_hat0 = tensor_hat0+tensor_hat;
    
    % Sample precision \tau_{\epsilon}.
    var_a = a0+0.5*length(position);
    error = sparse_tensor-tensor_hat;
    var_b = b0+0.5*sum(error(position).^2);
    tau_epsilon = gamrnd(var_a,1./var_b);
end
for k = 1:d
    factor_mat{k} = factor_mat{k}./iters;
end

tensor_hat = tensor_hat0/iters;
final_result = cell(2,1);
FinalMAPE = sum(abs(dense_tensor(pos)-tensor_hat(pos))./dense_tensor(pos))./length(pos);
final_result{1} = FinalMAPE;
FinalRMSE = sqrt(sum((dense_tensor(pos)-tensor_hat(pos)).^2)./length(pos));
final_result{2} = FinalRMSE;

% Print the results.
fprintf('Final RMSE = %g, MAPE = %g\n',FinalRMSE,FinalMAPE);
end

Created file 'C:\Program Files\MATLAB\R2016b\extern\engines\python\BGCP_Gibbs.m'.


In [9]:
[tensor_hat,model,result] = BGCP_Gibbs(tensor,sparse_tensor,'CP_rank',3,'maxiter',2000);


------Bayesian Gaussian CP decomposition using Gibbs sampling------
iteration = 1, RMSE = 151.01.
iteration = 2, RMSE = 48.4965.
iteration = 3, RMSE = 27.7547.
iteration = 4, RMSE = 24.2612.
iteration = 5, RMSE = 31.6578.
iteration = 6, RMSE = 26.5075.
iteration = 7, RMSE = 20.7942.
iteration = 8, RMSE = 21.618.
iteration = 9, RMSE = 27.0666.
iteration = 10, RMSE = 34.3827.
iteration = 11, RMSE = 32.053.
iteration = 12, RMSE = 29.0464.
iteration = 13, RMSE = 23.9818.
iteration = 14, RMSE = 37.9526.
iteration = 15, RMSE = 28.4163.
iteration = 16, RMSE = 28.4664.
iteration = 17, RMSE = 23.9012.
iteration = 18, RMSE = 31.8868.
iteration = 19, RMSE = 24.8787.
iteration = 20, RMSE = 24.6603.
iteration = 21, RMSE = 23.5096.
iteration = 22, RMSE = 26.6971.
iteration = 23, RMSE = 26.1986.
iteration = 24, RMSE = 20.4857.
iteration = 25, RMSE = 20.2202.
iteration = 26, RMSE = 15.4819.
iteration = 27, RMSE = 31.0592.
iteration = 28, RMSE = 31.0276.
iteration = 29, RMSE = 29.0842.
iteration = 30,

In [10]:
tensor_hat


tensor_hat(:,:,1) =

  169.7288   73.8806  493.2441  367.2362
  102.2117   47.9834  329.6397  343.4680
  169.7687   80.0803  546.8681  581.0881
  173.2416   80.5926  539.4201  550.4525
  136.5548   64.9221  496.7241  488.5281
   84.3753   39.9322  304.3534  295.0431
  124.9345   57.8355  386.5663  390.8482


tensor_hat(:,:,2) =

  163.2806   72.5756  577.2114  397.3160
  100.4564   46.8018  306.4529  327.5689
  167.0897   78.1510  502.8549  552.0994
  169.2941   78.4757  520.3087  532.1214
  139.7732   65.3666  441.2296  467.0010
   86.0696   40.1059  274.9461  283.4649
  121.8632   56.2621  375.2541  378.8582


tensor_hat(:,:,3) =

  217.6636   92.5035  448.2096  431.3107
   98.7862   47.0436  357.5936  347.7126
  161.9017   77.6208  594.7279  584.9739
  175.7903   82.1743  568.5081  564.3697
  121.2051   59.8375  574.3130  491.4519
   76.8258   37.5866  347.5208  298.6150
  128.2002   59.5646  405.5022  402.0577




In [11]:
sparse_tensor


sparse_tensor(:,:,1) =

   155    74   493     0
   108    44     0   359
     0    78   567   581
   181     0   517   552
   137    53   489   485
     0    44   306   290
   139    55   398   390


sparse_tensor(:,:,2) =

   172    69   590   386
     0    39   310   304
   158    74     0   546
     0    90   525   552
   150     0   438     0
     0    32   281     0
     0    51   358   382


sparse_tensor(:,:,3) =

   225    92   443   436
    94    44   355   356
   139    77   575   604
   175    98   574   553
   126     0   593   484
    58     0   348   301
   144    71     0   396




In [12]:
tensor


tensor(:,:,1) =

   155    74   493   426
   108    44   350   359
   175    78   567   581
   181   111   517   552
   137    53   489   485
    90    44   306   290
   139    55   398   390


tensor(:,:,2) =

   172    69   590   386
   104    39   310   304
   158    74   505   546
   176    90   525   552
   150    64   438   459
    73    32   281   299
   127    51   358   382


tensor(:,:,3) =

   225    92   443   436
    94    44   355   356
   139    77   575   604
   175    98   574   553
   126    67   593   484
    58    49   348   301
   144    71   444   396


